# CNN model for Fashion MNIST Dataset
Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. Zalando intends Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits.

## Content
Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255. The training and test data sets have 785 columns. The first column consists of the class labels (see above), and represents the article of clothing. The rest of the columns contain the pixel-values of the associated image.

- To locate a pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27. The pixel is located on row i and column j of a 28 x 28 matrix.
- For example, pixel31 indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below. 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from subprocess import check_output
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.callbacks import LearningRateScheduler
num_classes = 10
random_seed = 10

## Data Exploration

In [ ]:
train_df = pd.read_csv('/kaggle/input/fashion-mnist_train.csv',sep=',')
test_df = pd.read_csv('/kaggle/input/fashion-mnist_test.csv', sep = ',')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

Now it is observed that the first column is the label data and because it has 10 classes so it is going to have from 0 to 9.The remaining columns are the actual pixel data.Here as you can see there are about 784 columns that contain pixel data. Here each row is a different image representation in the form pixel data.

Now let us split the train data into x and y arrays where x represents the image data and y represents the labels.

To do that we need to convert the dataframes into numpy arrays of float32 type which is the acceptable form for tensorflow and keras.

In [ ]:
train_data = np.array(train_df, dtype = 'float32')
test_data = np.array(test_df,dtype='float32')

Divide the train and test data into label and data separately. 
x_train has all the column except label and header, same goes for x_test. y_train and y_test will have all the label for train and test.
We will divide x_train and x_test so that all the values are in from 0 to 1.

In [ ]:
x_train = train_data[:,1:]/255
y_train = train_data[:,0]
x_test = test_data[:,1:]/255
y_test = test_data[:,0]

# CNN Model
## Define the model
## Compile the model
## Train the model
Shape of the image which we will feed the model

In [ ]:
image_rows = 28

image_cols = 28

batch_size = 512

image_shape = (1,image_rows,image_cols)

Formatting on the x_train and others according to the image shape

In [ ]:
x_train = x_train.reshape(x_train.shape[0],*image_shape)
x_test = x_test.reshape(x_test.shape[0],*image_shape)

x_test.shape

# Define the model
Model has been taken from [here](https://www.kaggle.com/ankurshukla03/mnist-using-ensemble-cnn-0-997). Using ensemble of CNN for the training

In [ ]:
nn = 5
model = [0]*nn

for j in range(nn):
    model[j] = Sequential()
    model[j].add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(1,28,28), activation='relu',data_format='channels_first'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))
    
    model[j].add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))
    
    model[j].add(Conv2D(128, kernel_size = 4, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Flatten())
    model[j].add(Dropout(0.4))
    model[j].add(Dense(10, activation='softmax'))
    
    # Compile model
    model[j].compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Data Augmentation

In [ ]:
# With data augmentation to prevent overfitting
datagen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

## Train 5 CNNs

In [ ]:
history = [0] * nn
epochs = 50

# Fit the model
for j in range(nn):
    x_train2, x_val, y_train2, y_val = train_test_split(x_train, y_train, test_size = 0.10, random_state=random_seed)
    history[j] = model[j].fit_generator(datagen.flow(x_train2,y_train2, batch_size=64),
                              epochs = epochs, validation_data = (x_val,y_val),
                              verbose = 0, steps_per_epoch=(len(x_train)//64),validation_steps=(len(x_val)//64))
    print("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.2f}, Validation accuracy={3:.2f}".format(
        j+1,epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))

## Test Prediction Accuracy
Now we will calculate the test loss and accuracy. We can calculate the avg loss and accuracy

In [ ]:
score = [0]*nn

for j in range(nn):
    score[j] = model[j].evaluate(x_test,y_test)

In [ ]:
np_score = np.array(score)
# axis = 0 means calculating across columns
np_sum = np.sum(np_score,axis =0)
avg_loss = (np_sum[0]/len(np_score))
avg_accuracy = (np_sum[1]/len(np_score))
print('Avg test loss: ', avg_loss)
print('Avg test accuracy: ',avg_accuracy)